In [1]:
import nibabel as nib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
dataset_path = '/home/menishe/DM_Project/BET_BSE_DATA/'

Img_path=dataset_path+ 'files/'

In [23]:
dataset = pd.read_csv(dataset_path + "Label_file.csv")
dataset.head(6)

,Filename,Recognizable-Facial-Feature,Brain-Feature-Loss
0,IXI369-Guys-0924-T1_bet_03.nii,Yes,No
1,IXI448-HH-2393-T1_bet_07.nii,Yes,No
2,IXI252-HH-1693-T1_bet_08.nii,Yes,No
3,IXI188-Guys-0798-T1_bet_17.nii,Yes,No
4,IXI182-Guys-0792-T1_bet_17.nii,Yes,No
5,IXI635-HH-2691-T1_bet_12.nii,Yes,No


In [33]:
dataframe3 = dataset.iloc[:-1,:]
dataframe3['Recognizable-Facial-Feature'].value_counts()
dataframe3['Brain-Feature-Loss'].value_counts()
dataframe3.head(6)

,Filename,Recognizable-Facial-Feature,Brain-Feature-Loss
0,IXI369-Guys-0924-T1_bet_03.nii,Yes,No
1,IXI448-HH-2393-T1_bet_07.nii,Yes,No
2,IXI252-HH-1693-T1_bet_08.nii,Yes,No
3,IXI188-Guys-0798-T1_bet_17.nii,Yes,No
4,IXI182-Guys-0792-T1_bet_17.nii,Yes,No
5,IXI635-HH-2691-T1_bet_12.nii,Yes,No


In [34]:
from scipy import ndimage

def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath).get_fdata()
    return scan


def normalize(volume):
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    return volume.astype("float32")


def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img


def process_scan(path):
    """Read and resize volume"""
    # Read scan
    volume = read_nifti_file(path)
    # Normalize
    volume = normalize(volume)
    # Resize width, height and depth
    volume = resize_volume(volume)
    return volume

In [35]:
preprocess_data = np.array([process_scan(Img_path + filepath) for filepath in dataframe3['Filename']])


In [52]:
print("Train Size: ", round(1311 * 0.70))
print("Test Size: ", round(1311 * 0.30))


Train Size:  918
Test Size:  393


In [70]:
X_train = preprocess_data[:918, :]
X_test= preprocess_data[917:, :]

411041792

In [69]:
len(X_test)

393

In [55]:
#dataset["Recognizable-Facial-Feature"] = dataset["Recognizable-Facial-Feature"].replace({"Yes": 1, 'No': 0})
#dataset["Brain-Feature-Loss"] = dataset["Brain-Feature-Loss"].replace({"Yes": 1, 'No': 0})


In [56]:
X_test_facial = dataset['Recognizable-Facial-Feature'][:918]
X_test_facial = dataset['Recognizable-Facial-Feature'][918:]
X_test_brain = dataset['Brain-Feature-Loss'][:918]
X_test_brain = dataset['Brain-Feature-Loss'][918:]

In [71]:
with open('X_train.pickle', 'wb') as f:
    pickle.dump(X_train, f)
with open('X_test.pickle', 'wb') as f:
    pickle.dump(X_test, f)

In [58]:
dataset.iloc[:,:].to_csv('Binary_data.csv', index = False)
